# Alpha Test Harness

Test ideas relating to alpha racing performance optimisations

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os

import numpy as np

from math import pi, cos, sqrt

from common_core import Printable, projdir, loadCsv

## Constants

In [2]:
# Mean radius of Earth (m). IUGG and IERS both give a value of 6371008.7714 metres
EARTH_RADIUS = 6371009

# Exact number of metres in a nautical mile
METRES_PER_NM = 1852

# Minimum time betwen individual alphas (s) is only required when maintaining a list of alpha results
MIN_ALPHA_INTERVAL = 10

# Minimum alpha distance (m) is used to safely ignore a decent number of readings
MIN_ALPHA_DISTANCE = 200

# Maximim alpha distance (m) is one of the core parameters for alpha racing
MAX_ALPHA_DISTANCE = 500

# Minimum alpha proximity (m) is one of the core parameters for alpha racing
MAX_ALPHA_PROXIMITY = 50

# Minimum speed to be interested in a data point (m/s)
MIN_SPEED_FILTER = 1

# Minimum number of degrees to determine that a turn has occurred and thus an alpha may be in progress
MIN_TURN_DEGREES = 120

## Speedsurfing Session

Generic class for holding an entire speedsurfing session

In [3]:
class SpeedsurfingSession(Printable):
    
    def __init__(self, filename=None):
        '''Constructor just records the provided values'''
        
        self.track = None
        self.extras = None

        self.filename = filename
        
        if self.filename:
            self.loadTrack()


    def loadTrack(self, filename=None):
        '''Process the track'''

        if filename:
            self.filename = filename

        # Load the FIT data from the CSV
        self.track = loadCsv(self.filename)
        
        # Identify numpy arrays for later access
        self.indices = self.track['idx']
        self.speeds = self.track['speed']
        self.cogs = self.track['cog']
        
        # Pre-calculate some attributes so they can be cached / buffered
        self.populateCache()


    def populateCache(self):
        '''Populate the cache for speed calculations, including alphas'''

        # Convert timestamps into datetimes
        self.datetimes = np.datetime64('1989-12-31T00:00') + self.track['timestamp'].astype('timedelta64[s]')

        # Convert latitude and longitude values from semicircles to radians
        self.latitudes = np.radians(self.track['position_lat'] * (180.0 / 2 ** 31))
        self.longitudes = np.radians(self.track['position_long'] * (180.0 / 2 ** 31))

        # Calculate distances north / south of the equator (metres)
        self.yOffsets = self.latitudes * EARTH_RADIUS

        # Calculate the scaling factors for distances east / west of the primary meridian
        self.xScales = EARTH_RADIUS * np.cos(self.latitudes)

        # Calculate cumulative distances (m) using speed (m/s)
        self.distances = self.speeds.cumsum()

        # Apply speed filter (standard practice for GPSResults, GP3S, etc)
        self.filters = np.where(self.speeds < MIN_SPEED_FILTER, 1, 0)


    def processAlphas(self):
        '''Process the alphas'''

        # List of all unique alphas - separated by MIN_ALPHA_INTERVAL
        alphaResults = []
        
        # Current alpha being considered - required to keep track of the actual alpha, ignoring slower 1 second variants
        alphaResult = None
        
        # Endpoint of the most recent alpha that was considered
        alphaIdx = None

        # Performance monitoring
        proximityCheckCount = 0

        # Process GNSS data readings one at a time, thus simulating real-time processing (well, sort of)
        for i2 in range(len(self.track)):

            # Since we've passed the minimum alpha interval, store the previous alpha in the results array
            if alphaResult and i2 >= alphaIdx + MIN_ALPHA_INTERVAL:
                alphaResults.append(alphaResult)
                alphaResult = None

            # Skip alpha processing if latest speed is less than the filter value (2.5 m/s, ~5 knots)
            if self.filters[i2] == 0:

                # Processing will start with the previous reading, working backwards in time
                i1 = i2 - 1

                # Do not go beyond the max alpha distance (e.g. 500m)
                cutoff = self.distances[i2] - MAX_ALPHA_DISTANCE

                # Skip the most recent 200m, plus any readings that were flagged by the speed filter (e.g. <2.5 m/s)
                skip = self.distances[i2] - MIN_ALPHA_DISTANCE
                while i1 >= 0 and self.filters[i1] == 0 and self.distances[i1] > skip:
                    i1 -= 1

                # Look for start(s) of the alpha, aborting if filtered (e.g. <2.5 m/s) or the max distance (500 m) is reached
                while i1 >= 0 and self.filters[i1] == 0 and self.distances[i1] >= cutoff:

                    # Simple approach to find the difference between two COG values
                    angularDifference = 180 - abs(abs(self.cogs[i2] - self.cogs[i1]) - 180)

                    # An alpha is a "there and back" (includes a gybe or tack) so COG must be significantly different
                    if angularDifference >= MIN_TURN_DEGREES:

                        # Pythagoras estimate of proximity between two points is plenty good enough for 50m (< 1 mm accuracy)
                        proximity = estimateProximity(self.longitudes, self.yOffsets, self.xScales, i1, i2)
                        proximityCheckCount += 1

                        # Since this is a potential alpha it needs to be properly considered
                        if proximity <= MAX_ALPHA_PROXIMITY:

                            # Distance calculation is quick and easy
                            distance = self.distances[i2] - self.distances[i1]

                            # Should probably calculate seconds from the timestamps but this will suffice...
                            seconds = self.indices[i2] - self.indices[i1]

                            # Speed in knots, although could store in other units such as m/s, cm/s or mm/s
                            speed = distance / seconds * 3600 / METRES_PER_NM

                            # Create an alpha object which may be stored later
                            if alphaResult is None or speed > alphaResult.speed:
                                alphaResult = SpeedsurfingResult(
                                                    self.indices[i2], self.datetimes[i1], self.datetimes[i2],
                                                    distance, speed)

                            # Remember where this latest alpha actually finished, although there may be a better one later!
                            alphaIdx = i2

                    i1 -= 1

        # Store final alpha (if present)
        if alphaResult:
            alphaResults.append(alphaResult)

        # Sort and print alpha results
        if len(alphaResults) > 0:

            alphaResults.sort(key=lambda alphaResult: alphaResult.speed, reverse=True)

            print('start time\tend time\tm\tknots')

            for i in range(len(alphaResults)):
                print('{}\t{}\t{:0.1f}\t{:0.3f}'.format(
                                                    str(alphaResults[i].startTime)[-8:],
                                                    str(alphaResults[i].endTime)[-8:],
                                                    alphaResults[i].distance, alphaResults[i].speed))

            print()

        print('{} proximity checks'.format(proximityCheckCount))

## Speedsurfing Result

Generic class for holding a single speedsurfing run / result

In [4]:
class SpeedsurfingResult(Printable):
    
    def __init__(self, idx, startTime, endTime, distance, speed):
        '''Constructor just records the field values'''
        
        self.idx = idx
        self.startTime = startTime
        self.endTime = endTime
        self.distance = distance
        self.speed = speed

## Proximity Testing

Function to calculate the proximity between two nearby points. Utilises cache so no need for lots of costly trig functions!

See the 'haversine_vs_pythagoras' notebook for a full comparison of the Haversine Formula vs Pythagorean Theorem.

The code here is very similar to the aforementioned notebook but makes use of pre-calculated values, where appropriate.

In [5]:
def estimateProximity(longitudes, y_offsets, x_scales, i1, i2):
    '''Estimate the Euclidean distance between two nearby points on a sphere using Pythagorean Theorem'''

    # Calculate distance north / south
    yDelta = y_offsets[i2] - y_offsets[i1]

    # For the sake of completeness we need to cope with the points either side of the 180th meridian
    longDelta = abs(longitudes[i2] - longitudes[i1])
    if longDelta > pi:
        longDelta -= 2 * pi

    # Calculate distance east / west
    xDelta = longDelta * x_scales[i2]

    # Apply Pythagorean theorem to determine the distance, accurate to within the mm due to proximity check of 50m
    distance = sqrt(xDelta ** 2 + yDelta ** 2)

    return distance

## 20 Oct 2021

COROS APEX Pro and App both showed 23.62 knots, actually 21.632 knots when calculated correctly

In [6]:
if __name__ == '__main__':
    
    filename = os.path.join(projdir, 'sessions', '20211020', 'Speedsurfing20211020115946.csv')
    
    session = SpeedsurfingSession(filename=filename)

    session.processAlphas()

start time	end time	m	knots
11:49:26	11:50:09	478.5	21.632
12:40:55	12:41:40	494.0	21.338
13:52:51	13:53:37	497.5	21.022
12:21:41	12:22:27	497.1	21.004
11:11:54	11:12:40	489.3	20.675
11:13:17	11:14:04	499.0	20.640
12:37:39	12:38:27	487.4	19.737
12:25:55	12:26:44	493.2	19.566
13:45:47	13:46:34	466.1	19.278
14:20:39	14:21:29	492.3	19.140
11:28:32	11:29:23	499.7	19.046
12:17:43	12:18:33	489.8	19.041
13:04:23	13:05:13	489.7	19.037
12:49:59	12:50:51	493.7	18.457
11:25:16	11:26:05	463.0	18.368
13:29:44	13:30:28	414.4	18.309
14:15:54	14:16:47	498.7	18.290
11:31:48	11:32:41	496.9	18.224
12:32:57	12:33:46	458.8	18.202
14:18:25	14:19:18	490.8	18.002
13:56:09	13:57:04	497.8	17.594
11:52:42	11:53:06	204.1	16.534
14:22:41	14:23:43	457.9	14.358
14:27:40	14:29:01	499.9	11.997

29861 proximity checks


## 12 Nov 2021

COROS APEX Pro and App both showed 27.64 knots, actually 21.148 knots when calculated correctly 

In [7]:
if __name__ == '__main__':

    filename = os.path.join(projdir, 'sessions', '20211112', 'Speedsurfing20211112122819.csv')
    
    session = SpeedsurfingSession(filename=filename)
    
    session.processAlphas()

start time	end time	m	knots
12:47:38	12:48:23	489.6	21.148
12:37:07	12:37:54	491.9	20.344
12:39:19	12:40:07	499.9	20.242
13:07:25	13:08:13	497.8	20.158
12:31:32	12:32:21	491.9	19.515
13:11:21	13:12:10	491.1	19.480
13:19:28	13:20:18	491.9	19.124
12:43:17	12:44:10	492.8	18.074
13:23:33	13:24:28	495.0	17.493
12:55:33	12:56:30	492.4	16.794
12:35:04	12:35:54	413.5	16.075

19336 proximity checks


## 4 Apr 2022

COROS APEX Pro and App both showed 22.51 knots, actually 22.014 knots when calculated correctly

In [8]:
if __name__ == '__main__':

    filename = os.path.join(projdir, 'sessions', '20220404', 'Speedsurfing20220404134130.csv')
    
    session = SpeedsurfingSession(filename=filename)
    
    session.processAlphas()

start time	end time	m	knots
12:55:24	12:56:08	498.3	22.014
13:57:31	13:58:18	493.7	20.419
13:00:40	13:01:27	493.2	20.397
13:00:06	13:00:52	477.3	20.171
12:58:08	12:58:57	498.9	19.790
13:03:13	13:04:02	488.6	19.383
14:28:16	14:29:05	485.5	19.260
13:18:40	13:19:30	491.6	19.112
14:23:28	14:24:17	478.2	18.972
14:22:45	14:23:35	485.7	18.883
14:34:41	14:35:12	294.5	18.465
12:53:10	12:53:49	369.7	18.428
13:06:10	13:06:48	351.8	17.997
12:51:29	12:51:58	268.3	17.986
13:36:39	13:37:16	340.9	17.912
13:08:46	13:09:11	229.5	17.841
12:57:28	12:58:08	367.1	17.837
13:54:30	13:54:54	216.6	17.544
13:15:05	13:15:37	288.6	17.530
14:27:31	14:28:26	493.0	17.424
13:21:32	13:22:24	465.8	17.412
13:35:50	13:36:35	402.6	17.390
13:32:27	13:33:04	330.7	17.373
12:54:39	12:55:30	448.6	17.099
12:54:02	12:54:31	250.6	16.797
12:50:48	12:51:26	323.6	16.552
14:34:07	14:34:32	212.5	16.520
12:47:23	12:48:03	315.3	15.322
13:05:16	13:06:19	493.2	15.219
12:46:33	12:46:59	203.1	15.185
13:44:04	13:44:30	202.8	15.159
12:56:46	12